# Base blockchain -- trace data

By [Johnnatan Messias](https://johnnatan-messias.github.io), August 20th

### Base Chain Trace Columns

These columns likely come from Base chain and represent execution traces from transaction replays (e.g., via `debug_traceTransaction`). Below is a description of each column:

| Column           | Description                                                                |
| ---------------- | -------------------------------------------------------------------------- |
| `block_time`     | Timestamp of the block in human-readable format.                           |
| `block_number`   | The block number where the trace occurred.                                 |
| `value`          | Amount of ETH (in wei) transferred in the trace.                           |
| `gas`            | Gas limit specified for this trace.                                        |
| `gas_used`       | Actual gas consumed by the operation.                                      |
| `block_hash`     | The hash of the block containing this trace.                               |
| `success`        | Indicates if this specific trace step was successful (`True`/`False`).     |
| `tx_index`       | Transaction index within the block.                                        |
| `sub_traces`     | Number of sub-calls from this trace (e.g., internal contract calls).       |
| `error`          | Error message if the trace failed (e.g., 'revert').                        |
| `tx_success`     | Whether the entire transaction was successful.                             |
| `tx_hash`        | Hash of the transaction this trace belongs to.                             |
| `from`           | Address that initiated this trace.                                         |
| `to`             | Address receiving the call/message.                                        |
| `trace_address`  | Path within the transaction trace tree (e.g., `[0, 1]` for a nested call). |
| `type`           | Type of trace: `call`, `create`, `suicide`, etc.                           |
| `address`        | For `create` or `suicide`, the address being created/destroyed.            |
| `code`           | Contract bytecode (if available).                                          |
| `call_type`      | E.g., `call`, `delegatecall`, `staticcall`, etc.                           |
| `input`          | Hex-encoded input data to the contract.                                    |
| `output`         | Hex-encoded output returned by the call.                                   |
| `refund_address` | If ETH is being refunded (e.g., during contract destruction).              |
| `block_date`     | Date extracted from `block_time` (e.g., `YYYY-MM-DD`).                     |
| `to_label`       | Human-readable label for the `to` address, if available.                   |
| `from_label`     | Human-readable label for the `from` address, if available.                 |


In [1]:
# The "from" and "to" column is already the parsed traces gathered from the input.

In [2]:
import os
import json
import polars as pl

In [3]:
import sys
code_dir = os.path.realpath(os.path.join(os.getcwd(), "..", "src"))

sys.path.append(code_dir)

In [4]:
from utils import f_selector_dict, parse_tx_input

In [5]:
data_dir = os.path.realpath(os.path.join(os.getcwd(), "..", "data"))
os.makedirs(data_dir, exist_ok=True)

In [6]:
filedir = os.path.join(data_dir, "labels", "account-labels.json")
with open(filedir) as f:
    labels = json.load(f)
print("There are {} labels in our dataset.".format(len(labels)))

There are 501 labels in our dataset.


In [7]:
# Loading dead addresses
dead_addresses_df = pl.read_csv(os.path.join(
    data_dir, "labels", "dead_addresses.csv"))['address'].to_list()

In [8]:
# Loading arkhdam addresses
filedir = os.path.join(data_dir, "labels", "arkham_labels.csv")
arkham_labels = pl.read_csv(filedir)
arkham_labels.head()

address,chain,contract,tags_id,tags_label,tags_rank,tags_excludeEntities,tags_tagParams,arkhamLabel_name,arkhamLabel_chainType,name,note,id,type,service,addresses,website,twitter,crunchbase,linkedin
str,str,bool,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""0x1d42064fc4beb5f8aaf85f4617ae…","""arbitrum_one""",false,null,null,null,null,null,null,null,"""Uniswap""",null,"""uniswap""","""dex""",null,null,"""https://uniswap.org""","""https://twitter.com/Uniswap""","""https://www.crunchbase.com/org…","""https://www.linkedin.com/compa…"
"""0x1d42064fc4beb5f8aaf85f4617ae…","""avalanche""",false,null,null,null,null,null,null,null,"""Uniswap""",null,"""uniswap""","""dex""",null,null,"""https://uniswap.org""","""https://twitter.com/Uniswap""","""https://www.crunchbase.com/org…","""https://www.linkedin.com/compa…"
"""0x1d42064fc4beb5f8aaf85f4617ae…","""base""",false,null,null,null,null,null,null,null,"""Uniswap""",null,"""uniswap""","""dex""",null,null,"""https://uniswap.org""","""https://twitter.com/Uniswap""","""https://www.crunchbase.com/org…","""https://www.linkedin.com/compa…"
"""0x1d42064fc4beb5f8aaf85f4617ae…","""blast""",false,null,null,null,null,null,null,null,"""Uniswap""",null,"""uniswap""","""dex""",null,null,"""https://uniswap.org""","""https://twitter.com/Uniswap""","""https://www.crunchbase.com/org…","""https://www.linkedin.com/compa…"
"""0x1d42064fc4beb5f8aaf85f4617ae…","""bsc""",false,null,null,null,null,null,null,null,"""Uniswap""",null,"""uniswap""","""dex""",null,null,"""https://uniswap.org""","""https://twitter.com/Uniswap""","""https://www.crunchbase.com/org…","""https://www.linkedin.com/compa…"


In [9]:
schema = pl.Schema([('block_time', pl.String),
                    ('block_number', pl.Int64),
                    ('value', pl.Float64),
                    ('gas', pl.Int64),
                    ('gas_used', pl.Int64),
                    ('block_hash', pl.String),
                    ('success', pl.Boolean),
                    ('tx_index', pl.Int64),
                    ('sub_traces', pl.Int64),
                    ('error', pl.String),
                    ('tx_success', pl.Boolean),
                    ('tx_hash', pl.String),
                    ('from', pl.String),
                    ('to', pl.String),
                    ('trace_address', pl.String),
                    ('type', pl.String),
                    ('address', pl.String),
                    ('code', pl.String),
                    ('call_type', pl.String),
                    ('input', pl.String),
                    ('output', pl.String),
                    ('refund_address', pl.String),
                    ('block_date', pl.String)])

In [10]:
import ast
def selector_to_signature(sel: str) -> str:
    v = f_selector_dict.get(sel)
    if isinstance(v, dict):
        sig = v.get("signature")
        return sig if isinstance(sig, str) else ""
    return ""

filedir = os.path.join(data_dir, "base-revert-traces.csv")
df = pl.scan_csv(filedir, schema=schema)
df = df.with_columns(
    pl.col("block_time").str.strptime(
        pl.Datetime(), "%Y-%m-%d %H:%M:%S%.3f UTC"),
    to_label=pl.col("to").map_elements(
        lambda x: labels.get(x, x[:6]+"..."+x[-4:]), return_dtype=pl.String),
    from_label=pl.col("from").map_elements(
        lambda x: labels.get(x, x[:6]+"..."+x[-4:]), return_dtype=pl.String),
    function_selector=pl.col("input").str.slice(0, 10),
    trace_depth=pl.col("trace_address").map_elements(lambda x: 0 if (
        x == "" or x == "[]") else x.count(" ") + 1, return_dtype=pl.Int8)
).with_columns(function_selector_label=pl.col('function_selector').map_elements(selector_to_signature, return_dtype=pl.String))
# Group by transaction and sort by trace depth (to reconstruct call flow)
df = df.sort(["block_number", "tx_index", "tx_hash", "trace_depth"])

print("There are {} columns and {} rows.".format(
    len(df.collect_schema()), df.select(pl.len()).collect().item()))
df.head().collect()

There are 28 columns and 212052 rows.


block_time,block_number,value,gas,gas_used,block_hash,success,tx_index,sub_traces,error,tx_success,tx_hash,from,to,trace_address,type,address,code,call_type,input,output,refund_address,block_date,to_label,from_label,function_selector,trace_depth,function_selector_label
datetime[μs],i64,f64,i64,i64,str,bool,i64,i64,str,bool,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i8,str
2025-05-20 17:00:01,30485527,0.0,10500000,215430,"""0xcc299ee97244c721c649a8e18f10…",false,46,1,"""execution reverted""",false,"""0x6aaa7e785df914943d12dfb0b825…","""0xfe9d977baa824e683db0149cb80f…","""0xd1a0d188e861ed9d15773a2f3574…","""[]""","""call""",null,null,"""call""","""0x64ee4b8000000000000000000000…","""0x08c379a000000000000000000000…",null,"""2025-05-20""","""0xd1a0...ba8f""","""0xfe9d...640d""","""0x64ee4b80""",0,"""gm(address,uint8)"""
2025-05-20 17:00:01,30485527,0.0,10309879,188903,"""0xcc299ee97244c721c649a8e18f10…",false,46,16,"""execution reverted""",false,"""0x6aaa7e785df914943d12dfb0b825…","""0xd1a0d188e861ed9d15773a2f3574…","""0x5545a4ab5cd3f6e7db0883e73ea4…","""[0]""","""call""",null,null,"""delegatecall""","""0x64ee4b8000000000000000000000…","""0x08c379a000000000000000000000…",null,"""2025-05-20""","""0x5545...70a9""","""0xd1a0...ba8f""","""0x64ee4b80""",1,"""gm(address,uint8)"""
2025-05-20 17:00:01,30485527,null,10119325,8797,"""0xcc299ee97244c721c649a8e18f10…",false,46,1,null,false,"""0x6aaa7e785df914943d12dfb0b825…","""0xd1a0d188e861ed9d15773a2f3574…","""0x3e11001cfbb6de5737327c59e10a…","""[0 0]""","""call""",null,null,"""staticcall""","""0x9d63848a""","""0x0000000000000000000000000000…",null,"""2025-05-20""","""0x3e11...b5d3""","""0xd1a0...ba8f""","""0x9d63848a""",2,"""tokens()"""
2025-05-20 17:00:01,30485527,null,10093763,6297,"""0xcc299ee97244c721c649a8e18f10…",false,46,1,null,false,"""0x6aaa7e785df914943d12dfb0b825…","""0xd1a0d188e861ed9d15773a2f3574…","""0x3e11226fe3d85142b734abce6e58…","""[0 2]""","""call""",null,null,"""staticcall""","""0x9d63848a""","""0x0000000000000000000000000000…",null,"""2025-05-20""","""0x3e11...d1b4""","""0xd1a0...ba8f""","""0x9d63848a""",2,"""tokens()"""
2025-05-20 17:00:01,30485527,null,10083197,6309,"""0xcc299ee97244c721c649a8e18f10…",false,46,1,null,false,"""0x6aaa7e785df914943d12dfb0b825…","""0xd1a0d188e861ed9d15773a2f3574…","""0x3e1133ac082716ddc3114bbefeed…","""[0 3]""","""call""",null,null,"""staticcall""","""0x9d63848a""","""0x0000000000000000000000000000…",null,"""2025-05-20""","""0x3e11...9cb1""","""0xd1a0...ba8f""","""0x9d63848a""",2,"""tokens()"""


In [11]:
data_addresses = set(df.select('from').unique().collect()[
                     'from'].to_list() + df.select('to').unique().collect()['to'].to_list())
print("There are {} unique addresses".format(len(data_addresses)))

There are 4378 unique addresses


In [12]:
arkham_labels.filter(pl.col("address").is_in(data_addresses)
                     ).select(['address', 'name']).unique()

address,name
str,str
"""0xad01c20d5886137e056775af5691…",null
"""0x1111111254eeb25477b68fb85ed9…","""1inch"""
"""0x1231deb6f5749ef6ce6943a275a1…","""LiFi"""
"""0x111111125421ca6dc452d2893142…","""1inch"""
"""0xd7f1dd5d49206349cae8b585fcb0…","""Zerion"""
…,…
"""0x3a23f943181408eac424116af7b7…","""Socket"""
"""0xc35dadb65012ec5796536bd9864e…","""SushiSwap"""
"""0xba12222222228d8ba445958a75a0…","""Beets"""


In [13]:
df.select(pl.col("value").value_counts()).collect()

value
struct[2]
"{4.9571e15,2}"
"{2.0039e17,2}"
"{5.5254e11,2}"
"{4.2685e11,2}"
"{1.4526e11,2}"
…
"{3.3255e11,2}"
"{1.6468e11,2}"
"{4.5280e11,2}"


In [14]:
ast.literal_eval("[0 2]".replace(' ', ','))

[0, 2]

In [15]:
df.group_by("call_type").agg(pl.len()).sort(
    by="len", descending=True).collect()

call_type,len
str,u32
"""staticcall""",83323
"""call""",81544
"""delegatecall""",47184
null,1


In [16]:
df.group_by("error").agg(pl.len()).sort(
    by="len", descending=True).collect()

error,len
str,u32
null,182415
"""execution reverted""",29001
"""out of gas""",499
"""ReentrancySentryOOG""",56
"""out of gas: not enough gas for…",55
"""insufficient balance for trans…",12
"""write protection""",8
"""StateChangeDuringStaticCall""",3
"""out of gas: out of gas""",2


In [17]:
df.group_by("address").agg(pl.len()).sort(
    by="len", descending=True).collect()

address,len
str,u32
null,212052


In [18]:
# There are only failed traces
df.group_by("success").agg(pl.len()).collect()

success,len
bool,u32
false,212052


In [19]:
# There are only failed traces
df.group_by("tx_success").agg(pl.len()).collect()

tx_success,len
bool,u32
false,212052


In [20]:
df.group_by("from_label").agg(pl.len()).sort(
    by="len", descending=True).head(10).collect()

from_label,len
str,u32
"""0xb7b8...f463""",37058
"""USDC (USDC)""",22011
"""Uniswap V4: Pool Manager""",9172
"""0xad4f...915d""",4488
"""0x2a84...795b""",4433
"""KyberSwap: Aggregator Executor""",4426
"""0x4f1c...d037""",4016
"""0x5e7b...809a""",4003
"""0xdb45...6b5d""",2832


In [21]:
df.group_by("to_label").agg(pl.len()).sort(
    by="len", descending=True).head(10).collect()

to_label,len
str,u32
"""Circle""",22103
"""USDC (USDC)""",21997
"""Wrapped Ether (WETH)""",20714
"""Uniswap V4: Pool Manager""",18415
"""0xb7b8...f463""",10172
"""0xec8e...5831""",6855
"""0x5e7b...809a""",3129
"""Coinbase Wrapped BTC (cbBTC)""",1930
"""0x7458...aa72""",1929


In [22]:
df.group_by(["from_label", "to_label"]).agg(pl.len()).sort(
    by="len", descending=True).head(10).collect()

from_label,to_label,len
str,str,u32
"""USDC (USDC)""","""Circle""",21983
"""0xb7b8...f463""","""Uniswap V4: Pool Manager""",16370
"""0xb7b8...f463""","""Wrapped Ether (WETH)""",9763
"""0xb7b8...f463""","""USDC (USDC)""",9087
"""Uniswap V4: Pool Manager""","""0xb7b8...f463""",4131
"""Uniswap V4: Pool Manager""","""USDC (USDC)""",3915
"""Coinbase Wrapped BTC (cbBTC)""","""0x7458...aa72""",1929
"""0x5e7b...809a""","""Aerodrome: Voter""",1824
"""USD Base Coin (USDbC)""","""0x1833...b463""",1823


In [23]:
data = df.head(10).collect()
for row in data.iter_rows(named=True):
    print(row["tx_success"], row['tx_hash'],
          row["from_label"], "->", row["to_label"])

False 0x6aaa7e785df914943d12dfb0b825b5d43f196b96eb84b605d525405a027b697c 0xfe9d...640d -> 0xd1a0...ba8f
False 0x6aaa7e785df914943d12dfb0b825b5d43f196b96eb84b605d525405a027b697c 0xd1a0...ba8f -> 0x5545...70a9
False 0x6aaa7e785df914943d12dfb0b825b5d43f196b96eb84b605d525405a027b697c 0xd1a0...ba8f -> 0x3e11...b5d3
False 0x6aaa7e785df914943d12dfb0b825b5d43f196b96eb84b605d525405a027b697c 0xd1a0...ba8f -> 0x3e11...d1b4
False 0x6aaa7e785df914943d12dfb0b825b5d43f196b96eb84b605d525405a027b697c 0xd1a0...ba8f -> 0x3e11...9cb1
False 0x6aaa7e785df914943d12dfb0b825b5d43f196b96eb84b605d525405a027b697c 0xd1a0...ba8f -> 0x3e11...e6c9
False 0x6aaa7e785df914943d12dfb0b825b5d43f196b96eb84b605d525405a027b697c 0xd1a0...ba8f -> Pinto (PINTO)
False 0x6aaa7e785df914943d12dfb0b825b5d43f196b96eb84b605d525405a027b697c 0xd1a0...ba8f -> 0x3e11...b5d3
False 0x6aaa7e785df914943d12dfb0b825b5d43f196b96eb84b605d525405a027b697c 0xd1a0...ba8f -> 0x3e11...dbf1
False 0x6aaa7e785df914943d12dfb0b825b5d43f196b96eb84b605d525405a

In [24]:
column = "from"
data = (df
        .filter(pl.col('to_label').eq("Unknown") | pl.col("from_label").eq("Unknown"))
        .group_by(column)
        .agg(pl.len().alias('freq'))
        .sort(by='freq', descending=True).head(40).collect())
for row in data.iter_rows(named=True):
    print(row[column], row['freq'])

In [25]:
# Hex-encoded input data. The first 4 bytes (8 hex chars) are the function selector
(df
 .group_by("function_selector")
 .agg(pl.len())
 .sort(by="len", descending=True)
 .head(10)
 ).collect()

function_selector,len
str,u32
"""0x70a08231""",56944
"""0xa9059cbb""",21522
"""0x128acb08""",7903
"""0x3850c7bd""",7323
"""0x23b872dd""",6100
"""0xf135baaa""",4607
"""0x48c89491""",3724
"""0x91dd7346""",3453
"""0xf3cd914c""",3280


In [26]:
# Hex-encoded input data. The first 4 bytes (8 hex chars) are the function selector
(df
 .group_by("to_label")
 .agg(pl.len())
 .sort(by="len", descending=True)
 .head(10)
 ).collect()

to_label,len
str,u32
"""Circle""",22103
"""USDC (USDC)""",21997
"""Wrapped Ether (WETH)""",20714
"""Uniswap V4: Pool Manager""",18415
"""0xb7b8...f463""",10172
"""0xec8e...5831""",6855
"""0x5e7b...809a""",3129
"""Coinbase Wrapped BTC (cbBTC)""",1930
"""0x7458...aa72""",1929


In [27]:
# Hex-encoded input data. The first 4 bytes (8 hex chars) are the function selector
(df
 .group_by("from_label")
 .agg(pl.len())
 .sort(by="len", descending=True)
 .head(10)
 ).collect()

from_label,len
str,u32
"""0xb7b8...f463""",37058
"""USDC (USDC)""",22011
"""Uniswap V4: Pool Manager""",9172
"""0xad4f...915d""",4488
"""0x2a84...795b""",4433
"""KyberSwap: Aggregator Executor""",4426
"""0x4f1c...d037""",4016
"""0x5e7b...809a""",4003
"""0xdb45...6b5d""",2832


In [28]:
# Hex-encoded input data. The first 4 bytes (8 hex chars) are the function selector

data = (df
 .filter(pl.col('function_selector_label').ne(""))
 .group_by("function_selector_label")
 .agg(pl.len())
 .sort(by="len", descending=True)
 )

for row in data.collect().iter_rows(named=True):
    print(row['function_selector_label'], '\t', row['len'])

balanceOf(address) 	 56944
transfer(address,uint256) 	 21522
swap(address,bool,int256,uint160,bytes) 	 7903
slot0() 	 7323
transferFrom(address,address,uint256) 	 6100
exttload(bytes32) 	 4607
unlock(bytes) 	 3724
unlockCallback(bytes) 	 3453
swap((address,address,uint24,int24,address),(bool,int256,uint160),bytes) 	 3280
tickSpacing() 	 3006
uniswapV3SwapCallback(int256,int256,bytes) 	 2684
take(address,address,uint256) 	 2534
settle() 	 2522
liquidity() 	 2301
tickBitmap(int16) 	 2196
getFee(address) 	 2179
approve(address,uint256) 	 2123
getReserves() 	 1689
sync(address) 	 1681
withdraw(uint256) 	 1589
fee() 	 1476
deposit() 	 1385
latestRoundData() 	 1351
getSwapFee(address) 	 1316
pancakeV3SwapCallback(int256,int256,bytes) 	 1167
transfer_attention_tg_invmru_28108a2(bool,bool,uint256) 	 1156
observe(uint32[]) 	 1007
isAlive(address) 	 913
gauges(address) 	 912
executeBatchWithCallback((bytes,bytes)[],bytes) 	 625
getLatestPeriodInfo(address) 	 623
executeBatch((bytes,bytes)[],byte

In [29]:
input_test = "0x128acb08000000000000000000000000b7b8f759e8bd293b91632100f53a45859832f4630000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000d565b58619a8ccbb0000000000000000000000000000000000000000000342e8e317aa20c000000000000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000000000023078000000000000000000000000000000000000000000000000000000000000"
parse_tx_input(input_test, f_selector_dict)

{'selector': '0x128acb08',
 'known': True,
 'name': 'swap',
 'signature': 'swap(address,bool,int256,uint160,bytes)',
 'arg_types': ['address', 'bool', 'int256', 'uint160', 'bytes'],
 'args_decoded': ['0xB7B8F759E8Bd293b91632100f53a45859832f463',
  True,
  15376896090287164603,
  3942749655073213435084800,
  '0x3078']}

In [30]:
data = df.filter(pl.col('function_selector').eq('0x128acb08')).head(100)
for row in data.collect().iter_rows(named=True):
    print(f"tx_hash={row['tx_hash']}, input={row['input']}")

tx_hash=0x068fe17b56909063edc72a258e55ada673f83790f6cb05cbef839730780c058f, input=0x128acb080000000000000000000000006e4141d33021b52c91c28608403db4a0ffb50ec6000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000016c59fcd1607f00000000000000000000000000000000000000000000000000000001000276a400000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000000000e0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000400000000000000000000000006e4141d33021b52c91c28608403db4a0ffb50ec600000000000000000000000000000000000000000000000000000000000000600000000000000000000000007ddd5fc2588d58c0ebc7e8cfd3d94ee34cf85b230000000000000000000000004200000000000000000000000000000000000006000000000000000000000000d9aaec86b65d86f6a7b5b1b0c42ffa531710b6ca
tx_hash=0xb00fb519cedfde478e5fb26e0b66ece66d996cad307bd12a27197e1c82cdfd20,

In [31]:
data = (df
    .group_by('function_selector')
    .agg([pl.len().alias('count'), pl.col("input").first().alias("sample_input"), pl.col('function_selector_label').first(), pl.col('tx_hash').first()])
    .sort(by='count', descending=True)
    .head(50)
)
for row in data.collect().iter_rows(named=True):
    print(f"function_selector={row['function_selector']}, function_selector_label={row['function_selector_label']}, count={row['count']}, tx_hash={row['tx_hash']}, sample_input={row['sample_input']}")

function_selector=0x70a08231, function_selector_label=balanceOf(address), count=56944, tx_hash=0x6aaa7e785df914943d12dfb0b825b5d43f196b96eb84b605d525405a027b697c, sample_input=0x70a08231000000000000000000000000d1a0d188e861ed9d15773a2f3574a2e94134ba8f
function_selector=0xa9059cbb, function_selector_label=transfer(address,uint256), count=21522, tx_hash=0x068fe17b56909063edc72a258e55ada673f83790f6cb05cbef839730780c058f, sample_input=0xa9059cbb000000000000000000000000242f656b099a188bfc523bd56f84a6a9472d917400000000000000000000000000000000000000000000000000000000000f2228
function_selector=0x128acb08, function_selector_label=swap(address,bool,int256,uint160,bytes), count=7903, tx_hash=0x068fe17b56909063edc72a258e55ada673f83790f6cb05cbef839730780c058f, sample_input=0x128acb080000000000000000000000006e4141d33021b52c91c28608403db4a0ffb50ec6000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000016c59fcd1607f00000000000000000000000000000